In [ ]:
! pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 20.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("iwslt2017", "iwslt2017-en-de")

Generating train split:   0%|          | 0/206112 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8079 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/888 [00:00<?, ? examples/s]

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 206112
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 8079
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 888
    })
})


In [ ]:
train_data = dataset['train']
validation_data = dataset['validation']
test_data = dataset['test']

In [ ]:
print(type(train_data))
print()
print(train_data)
print()
print(train_data[0])

<class 'datasets.arrow_dataset.Dataset'>

Dataset({
    features: ['translation'],
    num_rows: 206112
})

{'translation': {'de': 'Vielen Dank, Chris.', 'en': 'Thank you so much, Chris.'}}


In [ ]:
import pandas as pd

train_data_dic = {
    'de': [],
    'en': []
}

for idx, en_de in enumerate(train_data):
  train_data_dic['de'].append(train_data[idx]['translation']['de'])
  train_data_dic['en'].append(train_data[idx]['translation']['en'])

# train dataset을 data frame 형태로 저장
# 각 행마다 'de', 'en' sentence pair
train_df = pd.DataFrame(train_data_dic)


In [ ]:
train_df

,de,en
0,"Vielen Dank, Chris.","Thank you so much, Chris."
1,"Es ist mir wirklich eine Ehre, zweimal auf die...",And it's truly a great honor to have the oppor...
2,Ich bin wirklich begeistert von dieser Konfere...,"I have been blown away by this conference, and..."
3,"Das meine ich ernst, teilweise deshalb -- weil...","And I say that sincerely, partly because I ne..."
4,Versetzen Sie sich mal in meine Lage!,Put yourselves in my position.
...,...,...
206107,Den alten Griechen fiel auch nicht eines Tages...,The Ancient Greeks didn't just wake up one day...
206108,"Wir Menschen brauchten Jahrhunderte, um Sachen...","It took centuries, even, for humans to realize..."
206109,Deshalb müssen wir unser Verständnis von Norma...,And so we must continuously challenge our noti...
206110,Figuren: Danke. Danke. Danke. Danke. Danke.,Characters: Thank you. Thank you. Thank you. T...


In [ ]:
! pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.3 MB/s eta 0:00:00


In [ ]:
import sentencepiece as spm

# 'de' sentence들로 구성된 txt
with open('de_train.txt', 'w', encoding='utf8') as f:
  f.write('\n'.join(train_df['de']))

# 'en' sentence들로 구성된 txt
with open('en_train.txt', 'w', encoding='utf8') as f:
  f.write('\n'.join(train_df['en']))

# bpe 활용해 de, en 각각 tokenize
# de.vocab de.model en.vocab en.model 생성됨
spm.SentencePieceTrainer.Train('--input=de_train.txt --model_prefix=de --model_type=bpe')
spm.SentencePieceTrainer.Train('--input=en_train.txt --model_prefix=en --model_type=bpe')

In [ ]:
import csv

de_vocab_list = pd.read_csv('de.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
en_vocab_list = pd.read_csv('en.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)

In [ ]:
de_vocab_list.sample(5)

,0,1
1561,▁Unternehmen,-1558
7253,dro,-7250
6007,▁stabil,-6004
6181,▁Wurz,-6178
7225,▁Chaos,-7222


In [ ]:
en_vocab_list.sample(5)

,0,1
1024,▁answ,-1021
4992,▁minim,-4989
3505,▁smell,-3502
2260,by,-2257
4383,▁dol,-4380


In [ ]:
print(f'de 단어집합 크기:  {len(de_vocab_list)}')
print(f'en 단어집합 크기: {len(en_vocab_list)}')

de 단어집합 크기:  8000
en 단어집합 크기: 8000


In [ ]:
sp = spm.SentencePieceProcessor()
sp.load('en.model') # en 모델 로드

True

In [ ]:
lines = [
  "I didn't at all think of it this way.",
  "I have waited a long time for someone to film"
]

for line in lines:
  print(line)
  print(sp.encode_as_pieces(line))
  print(sp.encode_as_ids(line))
  print()

I didn't at all think of it this way.
['▁I', '▁didn', "'", 't', '▁at', '▁all', '▁think', '▁of', '▁it', '▁this', '▁way', '.']
[35, 654, 7956, 7933, 134, 146, 227, 31, 60, 78, 288, 7953]

I have waited a long time for someone to film
['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film']
[35, 106, 1332, 763, 5, 613, 297, 91, 1278, 25, 2086]



In [ ]:
sp.GetPieceSize()

8000

In [ ]:
sp.IdToPiece(7134)

'▁proof'

In [ ]:
sp.PieceToId('▁character')

2428

In [ ]:
sp.DecodeIds([35, 106, 1332, 763, 5, 613, 297, 91, 1278, 25, 2086])

'I have waited a long time for someone to film'

In [ ]:
sp.DecodePieces(['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film'])

'I have waited a long time for someone to film'

In [ ]:
print(sp.encode('I have waited a long time for someone to film', out_type=str))
print(sp.encode('I have waited a long time for someone to film', out_type=int))

['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film']
[35, 106, 1332, 763, 5, 613, 297, 91, 1278, 25, 2086]


In [ ]:
validation_data_dic = {
    'de': [],
    'en': []
}

for idx, en_de in enumerate(validation_data):
  validation_data_dic['de'].append(validation_data[idx]['translation']['de'])
  validation_data_dic['en'].append(validation_data[idx]['translation']['en'])

# validation dataset을 data frame 형태로 저장
# 각 행마다 'de', 'en' sentence pair
validation_df = pd.DataFrame(validation_data_dic)

with open('de_validation.txt', 'w', encoding='utf8') as f:
  f.write('\n'.join(validation_df['de']))

with open('en_validation.txt', 'w', encoding='utf8') as f:
  f.write('\n'.join(validation_df['en']))

In [ ]:
test_data_dic = {
    'de': [],
    'en': []
}

for idx, en_de in enumerate(test_data):
  test_data_dic['de'].append(test_data[idx]['translation']['de'])
  test_data_dic['en'].append(test_data[idx]['translation']['en'])

# test dataset을 data frame 형태로 저장
# 각 행마다 'de', 'en' sentence pair
test_df = pd.DataFrame(test_data_dic)

with open('de_test.txt', 'w', encoding='utf8') as f:
  f.write('\n'.join(test_df['de']))

with open('en_test.txt', 'w', encoding='utf8') as f:
  f.write('\n'.join(test_df['en']))

In [ ]:
FOLDERNAME = 'backup'
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/MyDrive/{}'.format(FOLDERNAME))

# Change dariectory to current folder
%cd /content/drive/MyDrive/$FOLDERNAME

/content/drive/MyDrive/backup


In [ ]:
# train, validation, test dataset을 csv 형태로 저장
train_df.to_csv('train.csv', index=False)
validation_df.to_csv('validation.csv', index=False)
test_df.to_csv('test.csv', index=False)

In [ ]:
! ls

de.model      de_validation.txt  en_test.txt	    en.vocab   validation.csv
de_test.txt   de.vocab		 en_train.txt	    test.csv
de_train.txt  en.model		 en_validation.txt  train.csv
